In [52]:
import pandas as pd

# Read the CSV file
file_path = 'BTCUSDT-1m-2018-01.csv'  # Replace with your file path
column_names = ['time', 'open', 'high','low','close','volume','closetime','quote','trades','assest','q_quote','ignore']  # Replace with your actual column names

data = pd.read_csv(file_path,names=column_names)
data.head()
 # Set the stride for window movement
# data = data.drop(['time', 'open', 'high','low','volume','closetime','quote','trades','assest','q_quote','ignore'],axis=1)
# Implement the sliding window
data1 = data['close'].copy()
data = data1.head(500)
data


0      13707.92
1      13694.92
2      13680.00
3      13645.99
4      13600.00
         ...   
495    13610.27
496    13641.00
497    13681.01
498    13695.00
499    13689.64
Name: close, Length: 500, dtype: float64

In [38]:
import numpy as np
import tensorflow as tf

class TradingEnvironment:
    def __init__(self, prices):
        self.prices = prices
        # self.probabilities = probabilities
        self.current_step = 0
        self.balance = 10^7# Initial balance
        self.shares_held = 0
        self.max_steps = len(prices)-1
        self.state = 10^7
    def reset(self):
        self.current_step = 0
        self.balance = 10000000
        self.shares_held = 0
        
    def get_state(self):
        return ([self.prices[self.current_step], self.balance, self.shares_held])
    
    def take_action(self, action):
        current_price = self.prices[self.current_step]
        if action == 1:  # Buy
            if self.balance >= current_price:
                # Buy as many shares as possible with the available balance
                shares_to_buy = self.balance / current_price
                self.shares_held += shares_to_buy
                self.balance -= shares_to_buy * current_price
        elif action == -1:  # Sell
            if self.shares_held > 0:
                # Sell all shares held at once
                self.balance += self.shares_held * current_price
                self.shares_held = 0
        
    def step(self, action):
        self.take_action(action)
        self.current_step += 1
        if self.current_step >= self.max_steps:
            done = True
        else:
            done = False
        
        next_state = self.get_state()
        current_price = self.prices[self.current_step - 1]  # Get the current price

        # Print price and stock values at each step
        # print(f"Step: {self.current_step}, Price: {current_price}, Balance: {self.balance}, Shares Held: {self.shares_held}, action:{action}")


        reward = 0
        if self.current_step == self.max_steps:
            final_balance = self.balance + self.shares_held * self.prices[-1]

            if (self.balance == 0 and action == 1 ) and (self.shares_held == 0 and action == -1):
                reward = -e^(10)
            else:
                reward = final_balance -  1000  # Initial balance
            done = True
            print("Final_balance:",final_balance)
        
        return next_state, reward, done


In [39]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import random
class DQN(tf.keras.Model):
    def __init__(self, num_actions):
        super(DQN, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(64, activation='relu')
        self.output_layer = layers.Dense(num_actions, activation='linear')
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)
class DQNAgent:
    def __init__(self, num_actions, num_states):
        self.num_actions = num_actions
        self.num_states = num_states
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 8
        self.memory = []  # Experience replay memory
        self.model = DQN(num_actions)
        self.target_model = DQN(num_actions)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.update_target_network()
        self.model.compile(optimizer=self.optimizer, loss='mse')
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    def replay(self):
        for experience in self.memory:
            state, action, reward, next_state, done = experience
            print("state:",state,"experiance:",experience)
            if state is None or next_state is None:
                continue
            target = self.model.predict(np.array([state]))[0]
            if done:
                target[action] = reward
            else:
                next_state_target = self.target_model.predict(np.array([next_state]))[0]
                next_best_action = np.argmax(self.model.predict(np.array([next_state]))[0])
                target[action] = reward + self.gamma * next_state_target[next_best_action]
            self.model.train_on_batch(np.array([state]), np.array([target]))
    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice([-1,0,1])
        else:
            q_values = self.model.predict(np.array([state]))[0]
            return np.argmax(q_values)-1
    def train(self, env, num_episodes):
        for episode in range(num_episodes):
            state = env.reset()
            total_reward = 0
            while True:
                if state == None:
                    state = env.get_state()
                action = self.choose_action(state)
                next_state, reward, done = env.step(action)
                self.remember(state, action, reward, next_state, done)
                state = next_state
                # state = tf.convert_to_tensor(env.get_state(), dtype=tf.float32)
                total_reward += reward
                if done == True:
                    break
            if len(self.memory) > self.batch_size:
                self.replay()
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
            self.update_target_network()
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

In [54]:
prices = [13707.92,13694.92,13680,13645.99,13600,13568,13533.68 ,13530]
env = TradingEnvironment(prices)

# Create an instance of the DQNAgent
num_actions = 3  # Sell, Hold, Buy
num_states = len(prices)
agent = DQNAgent(num_actions, num_states)
num_episodes = 2  # Number of episodes for training
agent.train(env, num_episodes)
agent.model.save_weights('trained_model_weights.h5')


Final_balance: 9890350.877192982
Episode: 1, Total Reward: 9889350.877192982
Final_balance: 9915000.670526654
state: [13707.92, 10000000, 0] experiance: ([13707.92, 10000000, 0], -1, 0, [13694.92, 10000000, 0], False)
1/1 [==============================] - 0s 12ms/step


2023-12-12 13:13:06.657891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


state: [13694.92, 10000000, 0] experiance: ([13694.92, 10000000, 0], -1, 0, [13680, 10000000, 0], False)
1/1 [==============================] - 0s 40ms/step
state: [13680, 10000000, 0] experiance: ([13680, 10000000, 0], 1, 0, [13645.99, 0.0, 730.9941520467836], False)
1/1 [==============================] - 0s 18ms/step


2023-12-12 13:13:07.390686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:13:07.531037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


state: [13645.99, 0.0, 730.9941520467836] experiance: ([13645.99, 0.0, 730.9941520467836], 0, 0, [13600, 0.0, 730.9941520467836], False)
1/1 [==============================] - 0s 17ms/step
state: [13600, 0.0, 730.9941520467836] experiance: ([13600, 0.0, 730.9941520467836], 0, 0, [13568, 0.0, 730.9941520467836], False)
1/1 [==============================] - 0s 15ms/step
state: [13568, 0.0, 730.9941520467836] experiance: ([13568, 0.0, 730.9941520467836], 0, 0, [13533.68, 0.0, 730.9941520467836], False)
1/1 [==============================] - 0s 16ms/step


...

2023-12-12 13:13:08.423718: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:13:08.548652: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 0s 15ms/step
state: [13600, 0.0, 732.8160140817926] experiance: ([13600, 0.0, 732.8160140817926], 1, 0, [13568, 0.0, 732.8160140817926], False)
1/1 [==============================] - 0s 15ms/step
state: [13568, 0.0, 732.8160140817926] experiance: ([13568, 0.0, 732.8160140817926], 1, 0, [13533.68, 0.0, 732.8160140817926], False)
1/1 [==============================] - 0s 15ms/step


2023-12-12 13:13:08.682392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:13:08.806721: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 0s 14ms/step
state: [13533.68, 0.0, 732.8160140817926] experiance: ([13533.68, 0.0, 732.8160140817926], 1, 9914000.670526654, [13530, 0.0, 732.8160140817926], True)
1/1 [==============================] - 0s 13ms/step
Episode: 2, Total Reward: 9914000.670526654


2023-12-12 13:13:08.926371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:13:08.967230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


In [72]:
num_actions = 3  # Assuming 3 actions: -1, 0, 1
num_states = len(prices)
agent = DQNAgent(num_actions, num_states)

model = DQN(num_actions)
model.build = True

model.load_weights('trained_model_weights.h5')

current_state = [13707.92,13694.92,13680,13645.99,13600,13568,13533.68 ,13530]


# Use the trained agent to predict the action for the current state
predicted_action = agent.choose_action(current_state)

print("Predicted Action:", predicted_action)

ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [43]:
window_size = 10  # Set the size of the sliding window
stride = 1
for i in range(0, len(data)//5 - window_size + 1, stride):
    window = data.iloc[i:i+window_size]
    
    env = TradingEnvironment(np.array(window))

    # Create an instance of the DQNAgent
    num_actions = 3  # Sell, Hold, Buy
    num_states = len(prices)
    agent = DQNAgent(num_actions, num_states)
    # state = tf.convert_to_tensor(env.get_state(), dtype=tf.float32)
    
    # Train the DQN model
    num_episodes = 2  # Number of episodes for training
    agent.train(env, num_episodes)
    # Here, 'window' contains the data within the sliding window
    # You can perform operations on this window or pass it to your model for processing
    # Example: model.predict(window) or any other necessary operation
    
    # Printing the window for demonstration purposes
    print("Window:", i // stride + 1)
    print(window)

Final_balance: 9958202.373165093
state: [13707.92, 10000000, 0] experiance: ([13707.92, 10000000, 0], 0, 0, [13694.92, 10000000, 0], False)
1/1 [==============================] - 0s 13ms/step


2023-12-12 13:04:57.368818: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


state: [13694.92, 10000000, 0] experiance: ([13694.92, 10000000, 0], -1, 0, [13680.0, 10000000, 0], False)
1/1 [==============================] - 0s 12ms/step
state: [13680.0, 10000000, 0] experiance: ([13680.0, 10000000, 0], 0, 0, [13645.99, 10000000, 0], False)
1/1 [==============================] - 0s 13ms/step
state: [13645.99, 10000000, 0] experiance: ([13645.99, 10000000, 0], 1, 0, [13600.0, 0.0, 732.8160140817926], False)
1/1 [==============================] - 0s 12ms/step


2023-12-12 13:04:57.730755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:04:57.846670: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 0s 12ms/step
state: [13600.0, 0.0, 732.8160140817926] experiance: ([13600.0, 0.0, 732.8160140817926], 1, 0, [13568.0, 0.0, 732.8160140817926], False)
1/1 [==============================] - 0s 12ms/step
state: [13568.0, 0.0, 732.8160140817926] experiance: ([13568.0, 0.0, 732.8160140817926], -1, 0, [13533.68, 9942847.679061761, 0], False)
1/1 [==============================] - 0s 12ms/step


...

1/1 [==============================] - 0s 12ms/step
state: [13533.68, 10000000, 0] experiance: ([13533.68, 10000000, 0], 1, 0, [13530.0, 0.0, 738.8973287383772], False)
1/1 [==============================] - 0s 12ms/step
state: [13530.0, 0.0, 738.8973287383772] experiance: ([13530.0, 0.0, 738.8973287383772], -1, 0, [13575.0, 9997280.857830243, 0], False)
1/1 [==============================] - 0s 11ms/step


2023-12-12 13:05:11.751105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]
2023-12-12 13:05:11.864460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 0s 12ms/step
state: [13575.0, 9997280.857830243, 0] experiance: ([13575.0, 9997280.857830243, 0], 1, 0, [13554.58, 0.0, 736.4479453281947], False)
1/1 [==============================] - 0s 12ms/step


2023-12-12 13:05:11.965356: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1,3]
	 [[{{node Placeholder/_1}}]]


KeyboardInterrupt: 